# ⚛️ LiH — Noiseless VQE Ground-State Simulation

This notebook computes the **ground state energy of lithium hydride (LiH)**  
using a **noiseless Variational Quantum Eigensolver (VQE)**.

---

## **Simulation Summary**

- Molecule: **LiH**  
- Basis: **STO-3G**  
- Electrons: **4**  
- Ansatz: **UCC Doubles** (double excitations only)  
- Optimizer: **GradientDescent**  
- Device: **default.qubit** (statevector)  

### **Outputs**
- Convergence of VQE energy vs iteration  
- Final optimized parameters  
- Ground state decomposition into significant basis states  
- Bar plot of amplitudes  

This notebook follows the same structure and style as the H₂/H₃⁺ VQE notebooks.

In [1]:
# Core scientific stack
import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem
import matplotlib.pyplot as plt

import sys, os, json
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

# Shared project utilities
from vqe.vqe_utils import (
    IMG_DIR,
    excitation_ansatz,
    get_optimizer,
    set_seed,
    make_run_config_dict,
    run_signature,
    find_existing_run,
    save_run_record,
    ensure_dirs,
    build_run_filename,
)

ensure_dirs()

# 🧬 Part 1 — Molecular Setup

We construct the LiH molecular Hamiltonian in the STO-3G basis and prepare:

- The **Hartree–Fock** reference state  
- The **UCC doubles** excitation list  

LiH in STO-3G requires more qubits than H₂, but the UCC doubles ansatz helps
keep parameter count manageable.

In [2]:
seed = 0
set_seed(seed)

symbols = ["Li", "H"]
coordinates = np.array([
    [0.0, 0.0, 0.0],   # Li
    [0.0, 0.0, 1.6],   # H
])

basis = "STO-3G"
electrons = 4   # Li (3) + H (1)

optimizer_name = "GradientDescent"
stepsize = 0.2
max_iterations = 50
ansatz_desc = "UCC Doubles"

# Build Hamiltonian
hamiltonian, qubits = qchem.molecular_hamiltonian(
    symbols,
    coordinates,
    charge=0,
    basis=basis,
)

print(f"Number of qubits: {qubits}")

# Hartree–Fock state
hf = qchem.hf_state(electrons, qubits)
print("HF state =", "|" + "".join(str(b) for b in hf) + "⟩")

num_wires = qubits

# Excitations
singles, doubles = qchem.excitations(electrons, qubits)
print(f"Singles: {len(singles)}, Doubles: {len(doubles)}")

Number of qubits: 12
HF state = |111100000000⟩
Singles: 16, Doubles: 76


# 🧱 Part 2 — Noiseless VQE QNode  

We now build a **noiseless statevector QNode**:

- Applies a **UCC double-excitation ansatz**  
- Measures:

\[
E(\theta) = \langle \psi(\theta) | H | \psi(\theta) \rangle
\]

We provide both:
- a version that returns the **energy** (for optimization)  
- a version that returns the **full statevector**  

In [3]:
def get_vqe_qnode(return_state=False):
    """Create a noiseless QNode for LiH using the UCC doubles ansatz."""
    dev = qml.device("default.qubit", wires=num_wires)

    @qml.qnode(dev, diff_method="parameter-shift")
    def circuit(params):
        excitation_ansatz(
            params,
            wires=range(num_wires),
            hf_state=hf,
            excitations=doubles,
            excitation_type="double",
        )
        return qml.state() if return_state else qml.expval(hamiltonian)

    return circuit

# 💾 Part 3 — Cache-Aware VQE Wrapper

This wrapper:
- builds run configuration metadata  
- checks for cached results  
- or performs VQE from scratch  
- stores:
  - energy trajectory  
  - final parameters  
  - reproducible config hash  

This matches all other notebooks in your VQE suite.

In [4]:
def run_vqe_with_cache(seed=0):
    set_seed(seed)

    cfg = make_run_config_dict(
        symbols=symbols,
        coordinates=coordinates,
        basis=basis,
        ansatz_desc=ansatz_desc,
        optimizer_name=optimizer_name,
        stepsize=stepsize,
        max_iterations=max_iterations,
        seed=seed,
        noisy=False,
    )

    sig = run_signature(cfg)
    fname = build_run_filename("LiH_Noiseless", optimizer_name, seed, sig)
    existing = find_existing_run(sig)

    # --- Load cached run if available ---
    if existing:
        with open(existing) as f:
            rec = json.load(f)

        print(f"[reuse] Loaded {existing}")

        energies = rec.get("energies", [])

        # Parameter loading (new + legacy formats)
        if "final_params" in rec:
            params = np.array(rec["final_params"])
        elif "params_by_step" in rec and rec["params_by_step"]:
            params = np.array(rec["params_by_step"][-1])
        else:
            print("[reuse] Missing parameters — recomputing.")
            existing = None

        if existing:
            state_fn = get_vqe_qnode(return_state=True)
            final_state = state_fn(params)
            return energies, params, final_state

    # --- Fresh VQE run ---
    cost_fn = get_vqe_qnode(return_state=False)
    opt = get_optimizer(optimizer_name, stepsize=stepsize)

    params = np.zeros(len(doubles))
    energies = [float(cost_fn(params))]

    for _ in range(max_iterations):
        params, e_prev = opt.step_and_cost(cost_fn, params)
        energies.append(float(e_prev))

    # Final evaluation
    energies[-1] = float(cost_fn(params))

    # Save record
    record = {
        "config": cfg,
        "energies": energies,
        "final_params": params.tolist(),
    }
    save_run_record(fname, record)
    print(f"[saved] {optimizer_name}: {fname}")

    state_fn = get_vqe_qnode(return_state=True)
    final_state = state_fn(params)

    return energies, params, final_state

# 🚀 Part 4 — Run Noiseless VQE for LiH

In [5]:
energies, params, final_state = run_vqe_with_cache(seed=seed)

KeyboardInterrupt: 

# 📉 Part 5 — VQE Convergence Plot

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(energies, marker="o")
plt.xlabel("Iteration")
plt.ylabel("Energy (Ha)")
plt.title("LiH VQE Convergence (Noiseless, UCC Doubles + GradientDescent)")
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{IMG_DIR}/LiH_Noiseless_Convergence.png", dpi=300)
plt.show()

print(f"Final ground state energy: {energies[-1]:.8f} Ha")
if len(params) >= 2:
    print(f"First parameters: {params[0]:.6f}, {params[1]:.6f}")

# 🔍 Part 6 — Ground-State Reconstruction

We extract the final statevector and show only basis states with  
amplitude magnitude greater than:

\[
|\psi_i| > 10^{-2}
\]

In [ ]:
threshold = 1e-2

nonzero_idx = np.where(np.abs(final_state) > threshold)[0]
amps = final_state[nonzero_idx]

ket_terms = []
for idx, amp in zip(nonzero_idx, amps):
    bits = f"|{idx:0{num_wires}b}⟩"
    if abs(amp.imag) < 1e-6:
        amp_str = f"{amp.real:.4f}"
    else:
        amp_str = f"({amp.real:.4f} + {amp.imag:.4f}j)"
    ket_terms.append(f"{amp_str}{bits}")

ket_str = " + ".join(ket_terms)
print(f"|ψ⟩ (significant components):\n{ket_str}")

# 📊 Part 7 — Amplitude Bar Plot

In [ ]:
labels = [f"|{i}⟩" for i in nonzero_idx]
magnitudes = np.abs(amps)

plt.figure(figsize=(10,5))
plt.bar(labels, magnitudes)
plt.xlabel("Basis state")
plt.ylabel("Amplitude")
plt.title("LiH VQE Ground-State Amplitudes (Noiseless)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig(f"{IMG_DIR}/LiH_Ground_State.png", dpi=300)
plt.show()